# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
# Ваше решение

df['Соответствует нормативу'] = ''

for i, norma in enumerate(df['Норматив']):
    value = df.loc[i]['Результат анализа'].replace(',', '.')
    if value == 'б/цвета':
        value = '0'
    value = float(value)
    if '-' in norma:        
        threshold = [float(x) for x in norma.replace(',', '.').strip().split()[2].strip().split('-')]
    else:
        threshold = float(norma.replace(',', '.').strip().split()[2])
    if type(threshold) == list:
        if 'не более' in norma:
            df.loc[i]['Соответствует нормативу'] = (value <= threshold[1])
        else:
            df.loc[i]['Соответствует нормативу'] = (threshold[0] <= value <= threshold[1])
    else:
        df.loc[i]['Соответствует нормативу'] = (value <= threshold)

df

,Показатель,Единица измерений,Результат анализа,Норматив,Соответствует нормативу
0,pH,единицы pH,8.4,в пределах 6-9,True
1,Запах,баллы,1,не более 2-3,True
2,Цветность,градусы,б/цвета,не более 30,True
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,True
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",True
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",True
6,Нитраты (по NO3),мг/дм3,24,не более 45,True
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",True
8,Хлориды (Cl),мг/дм3,200,не более 350,True
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [4]:
# Ваше решение
box = ['orange'] * 5 + ['apple'] * 4
box
np.random.shuffle(box)
box

['orange',
 'orange',
 'apple',
 'orange',
 'orange',
 'orange',
 'apple',
 'apple',
 'apple']

In [5]:
def problem_2_experiment(box: list, iterations: int, series_count: int, item: str) -> float:
    iteration_results = []
    
    for _ in range(iterations):
        np.random.shuffle(box)
        iteration_results.append(box[:3].count(item) == series_count)
        
    probability = sum(iteration_results)/iterations
    return probability
        

In [7]:
result = []
for _ in range(10):
    experiment_probability = round(problem_2_experiment(box, iterations=1000000, series_count=3, item='orange'), 3)
    theoretical_probability = 0.119
    result.append(experiment_probability == theoretical_probability)
    
print(sum(result)/len(result))

0.8


###### Ответ: подтверждается, но только при большом количестве экспериментов. Экспериментальная и теоретическая вероятности должны сойтись при бесконечном количестве повторов.

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [8]:
# Ваше решение
parts_box = ['standard_part'] * 7 + ['not_standard_part'] * 3
np.random.shuffle(parts_box)
parts_box

['standard_part',
 'not_standard_part',
 'standard_part',
 'standard_part',
 'not_standard_part',
 'not_standard_part',
 'standard_part',
 'standard_part',
 'standard_part',
 'standard_part']

In [9]:
def problem_3_experiment(box: list, iterations: int, series_count: int, item: str) -> float:
    iteration_results = []
    
    for _ in range(iterations):
        np.random.shuffle(box)
        iteration_results.append(box[0] == item and box[1] != item)
        
    probability = sum(iteration_results)/iterations
    return probability

In [10]:
# 0.01 при range(1000) theoretical_probability = 0.23333
# 0.67, 0.61, 0.55, 0.66, 0.6 при range(100) theoretical_probability = 0.233 и round(..., 3)
result = []
for _ in range(10):
    experiment_probability = round(problem_3_experiment(parts_box, iterations=1000000, series_count=2, item='not_standard_part'), 5)
    theoretical_probability = 0.23333
    print(f'Экспериментальная вероятность: {experiment_probability}')
    result.append(experiment_probability == theoretical_probability)
    
print('\n', sum(result)/len(result)) 

Экспериментальная вероятность: 0.23345
Экспериментальная вероятность: 0.23386
Экспериментальная вероятность: 0.23387
Экспериментальная вероятность: 0.23331
Экспериментальная вероятность: 0.23323
Экспериментальная вероятность: 0.23286
Экспериментальная вероятность: 0.23349
Экспериментальная вероятность: 0.23341
Экспериментальная вероятность: 0.23367
Экспериментальная вероятность: 0.23336

 0.0


###### Ответ: скорее да, но потребуется большее количество экспериментов, чем для задачи 2. Экспериментальная и теоретическая вероятности должны сойтись при бесконечном количестве повторов. Интересно, что большое влияние оказывает с какой точностью надо считать. Предполагаю, что чем выше точность теоретической вероятности, тем большее количество экспериментов надо провести.